# Оценка потенциального экономического эффекта от внедрения полученного решения

Постройте простую экономическую модель для оценки эффекта от внедрения полученного решения на практике.
Например, введите следующие параметры:
- сколько денег в среднем приносит один пользователь в месяц;
- сколько денег в среднем вы будете вкладывать в удержание одного пользователя;
- с какой вероятностью пользователь примет ваше предложение;
- сколько пользователей (например, топ 1% или топ 25% согласно ранжированию по вашей модели) будет участвовать в кампании.

In [1]:
import pandas as pd
import random
import numpy as np
from scipy.sparse import coo_matrix, hstack
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest

from sklearn.metrics import roc_curve, precision_recall_curve, f1_score, roc_auc_score, recall_score, precision_score, log_loss, make_scorer

from sklearn.ensemble import GradientBoostingClassifier

seed = 1903
first_categorial_index = 190

In [2]:
month_revenue_per_user = 30 # Сколько денег в среднем приносит один пользователь в месяц (30 тугриков)
price_per_user_churn = 5 # Сколько денег будем вкладывать в удержание пользователя (5 тугриков)
retention_probability = 0.8 # Вероятность, с которой пользователь, собирающийся уйти, примет предложение
participants_perc = 0.1 # Сколько пользователей участвует в кампании.

## 1.
Введите еще несколько параметров и на их основе постройте простую экономическую модель: формулу по которой в зависимости от значения исходных параметров вы сможете рассчитать экономический эффект от проведения кампании (= сколько денег вы получили (или наоборот потеряли) от проведения кампании по удержанию) с использование вашей модели.

In [3]:
churn_size = 0.0744 # Процент пользователей, собирающихся уйти
recall = 0.2 # Полнота модели
clients_count = 12000 # Количество клиентов

def calculate_simple_revenue(participants_perc, recall, churn_size):
    churn_count = np.round(clients_count * churn_size, 0) # Количество пользователей, которые собриаются уйти
    churn_found = np.round(churn_count*recall, 0) # Количество пользователей, собирающихся уйти, которых выявила модель
    participants_count = np.round(participants_perc * clients_count, 0) # Количество пользователей в кампании
    not_participants_cnt = clients_count-participants_count # Количество пользователей не участвующих в кампании
    others_count = participants_count - churn_found # Количество пользователей не собирающихся уходить
    
    retention_size = np.round(churn_found * retention_probability, 0) # Количество пользователей, которых удалось сохранить
    churned = churn_count-retention_size # Количество пользователей, которые ушли
    saved_count = clients_count - churned # Количество пользователей после проведения кампании
    
    month_revenue = saved_count*month_revenue_per_user # Сколько денег получили в месяц
    money_spent = participants_count * price_per_user_churn # Сколько денег потратили на удержание в месяц
    
    month_revenue_default = (clients_count - churn_count) * month_revenue_per_user # Сколько денег получили бы если не проводили
                                                                                   # кампанию.
    campaign_revenue = month_revenue - money_spent
    return np.round(participants_count/clients_count*100, 2), campaign_revenue-month_revenue_default, retention_size

In [4]:
def build_simple_revenue_table():
    data = []
    for perc in np.arange(0.0, 1.0, 0.01):
        revenue_info = calculate_simple_revenue(perc, recall, churn_size)
        data.append(revenue_info)
    return data

## 2.

Исходя из введенных параметров проанализируйте, какой топ пользователей, склонных к оттоку согласно модели, оптимально выбрать для проведения кампании по удержанию для таким образом, чтобы экономический эффект был наибольшим? 

In [5]:
simple_revenue_table = build_simple_revenue_table()
print(pd.DataFrame(
    sorted([rd for rd in simple_revenue_table if rd[2] > 0], key=lambda r: -r[2]),
    columns=["Clients %", "Effect", "Saved clients count"]).head(5))
print()
print(pd.DataFrame(
    sorted(simple_revenue_table, key=lambda r: -r[1]),
    columns=["Clients %", "Effect", "Saved clients count"]).head(5))

   Clients %  Effect  Saved clients count
0        0.0  4290.0                143.0
1        1.0  3690.0                143.0
2        2.0  3090.0                143.0
3        3.0  2490.0                143.0
4        4.0  1890.0                143.0

   Clients %  Effect  Saved clients count
0        0.0  4290.0                143.0
1        1.0  3690.0                143.0
2        2.0  3090.0                143.0
3        3.0  2490.0                143.0
4        4.0  1890.0                143.0


По данной экономической модели можно увидеть, что проще вообще не поводить никакой кампании. Будем думать, что это связано с излишней простотой модели.

## 3.
Попробуйте усложнить экономическую модель. Добавьте еще несколько параметров и поиграйте с ними (например, измените стоимость удержания и вероятность, с которой пользователь принимает предложение), проанализируйте как меняется оптимальный размер топа?

Количество пользователей, которое будет участвовать в кампании будем считать исходя из значения порога, по которому строим предсказания.

Для этого придется построить модель и вероятности принадлежности клиентов из тестовой выборки к классу "отток".

Объявим функции, необходимые для построения модели

In [6]:
def fill_numericna(train_frame, test_frame, averageCalculator):
    """ Функция заполняет значения в числовом фрейме значениями, посчитанными averageCalculator. """
    
    # Посчитаем средние по колонкам
    numeric_avgs = averageCalculator(train_frame)
    
    # Оставим только те колонки, в которых среднее значение не равно NaN, т.к. в таких колонках совсем нет значений
    numeric_avgs = numeric_avgs.dropna()
    dropped_columns = train_frame.columns.drop(numeric_avgs.index)
    n_frame_train = train_frame[list(numeric_avgs.index)]
    n_frame_test = test_frame[list(numeric_avgs.index)]
    
    # Заполним пропущенные численные значения средними
    n_frame_train = n_frame_train.fillna(numeric_avgs, axis=0)
    n_frame_test = n_frame_test.fillna(numeric_avgs, axis=0)
    return (n_frame_train, n_frame_test, dropped_columns)

def fill_numericna_means(train_frame, test_frame):
    """ Функция заполняет значения в числовом фрейме средними и удаляет те колонки, в которых значений нет. """
    return fill_numericna(
        train_frame,
        test_frame,
        lambda f: f.mean(axis=0, skipna=True))

def fill_categorial_nav(train_frame, test_frame):
    return train_frame.fillna("NaV"), test_frame.fillna("NaV")

def remove_constant_features(frame, min_count=2):
    """Функция удаляет колонки, которые содержат только одно значение."""
    
    # Посчитаем количества уникальных значений по колонкам
    unique_counts = frame.nunique()
    # Удалим колонки с количеством значений меньшим min_count
    columns_to_drop = unique_counts[unique_counts < min_count].index
    
    return (frame.drop(columns=columns_to_drop), columns_to_drop)

class MatrixLabelEncoder:
    """ Класс кодирует категории числами от 0 до n, где n количество категорий в колонке. """
    
    def __init__(self):
        self.encoders = []
    
    def fit(self, matrix):
        for column_number in range(matrix.shape[1]):
            column = matrix[:,column_number]
            labelEncoder = LabelEncoder().fit(column)
            self.encoders.append(labelEncoder)
        return self
    
    def transform(self, matrix):
        transformed = np.empty(matrix.shape)
        for column_number in range(matrix.shape[1]):
            labelEncoder = self.encoders[column_number]
            num_column = labelEncoder.transform(matrix[:,column_number])
            for row_number, val in enumerate(num_column):
                transformed[row_number, column_number] = val
        return transformed

def predict_model_proba(X, model):
    """ Функция возвращает вероятности предсказаний для класса churn """
    return list(zip(*model.predict_proba(X)))[1]

def cleanup_frame_common(frame, numeric_features, categorial_features):
    """Функция делит признакина числовые и категориальные и удаляет константные признаки, содержащие только одно значение"""
    # Разделим коллекции на группы - числовые и категориальные.
    numeric_frame = frame[numeric_features].copy()
    categorial_frame = frame[categorial_features].copy()
    # Удалим вещественные колонки, содержащие одно и менее значений. 0 значений мы получаем, когда значения во всех строках Nan.
    numeric_frame_no_const, dropped_const_numeric_columns = remove_constant_features(numeric_frame)
    
    # Удалим категориальные колонки, содержащие ноль значений. Если есть одно значение, то могут быть Nan, которые для
    # категориальных признаков могут быть еще одной категорией (зависит от стратегии обработки).
    categorial_frame_no_const, dropped_const_categorial_columns = remove_constant_features(categorial_frame, 1)
    
    # Восстановим фрейм и вернем вместе с ним список удаленных категориальных колонок.
    return (pd.concat([numeric_frame, categorial_frame], axis=1),
            list(dropped_const_numeric_columns),
            list(dropped_const_categorial_columns))

def process_frame(
    train_frame,
    train_labels,
    test_frame,
    test_labels,
    numeric_features,
    categorial_features,
    fill_na_numerics,
    fill_na_categorial):
    """ Функция обрабатывает числовые признаки, заполняя пропуски. """
    
    # Удалим константные колонки из train_frame, и такие-же колонки из test_frame
    train_frame, const_numeric_columns, const_categorial_columns = cleanup_frame_common(
        train_frame,
        numeric_features,
        categorial_features)
    test_frame = test_frame.drop(columns=const_numeric_columns)
    test_frame = test_frame.drop(columns=const_categorial_columns)
    
    numeric_features = numeric_features.drop(const_numeric_columns)
    categorial_features = categorial_features.drop(const_categorial_columns)
    
    # Заполним пропущенные вещественные значения
    numeric_train, numeric_test, dropped_numeric = fill_na_numerics(
        train_frame[numeric_features],
        test_frame[numeric_features])
    
    numeric_features = numeric_features.drop(dropped_numeric)
    
    # Заполним пропущенные категориальные значения строками "NaV" (Not a value)
    categorial_train, categorial_test = fill_na_categorial(train_frame[categorial_features], test_frame[categorial_features])
    
    # Удалим категориальные колонки с одним единственным значением
    categorial_train, dropped_categorial = remove_constant_features(categorial_train)
    categorial_test = categorial_test.drop(columns=dropped_categorial)
    
    categorial_features = categorial_features.drop(dropped_categorial)
    
    # Список удаленных колонок
    dropped_numeric = np.concatenate([
        list(const_numeric_columns),
        list(dropped_numeric)])
    dropped_categorial = np.concatenate([
        list(const_categorial_columns),
        list(dropped_categorial)])
    
    return (pd.concat([numeric_train, categorial_train], axis=1),
            train_labels,
            pd.concat([numeric_test, categorial_test], axis=1),
            test_labels,
            dropped_numeric,
            dropped_categorial)

def process_frame_base(
    train_frame,
    train_labels,
    test_frame,
    test_labels,
    numeric_features,
    categorial_features):
    return process_frame(
        train_frame,
        train_labels,
        test_frame,
        test_labels,
        numeric_features,
        categorial_features,
        fill_numericna_means,
        fill_categorial_nav)

def scale_features(train_frame, test_frame):
    train_numeric = train_frame.as_matrix()
    
    scaler = StandardScaler().fit(train_numeric)
    
    train_numeric = coo_matrix(scaler.transform(train_numeric))
    test_numeric = coo_matrix(scaler.transform(test_frame.as_matrix()))
    
    return (train_numeric, test_numeric, scaler)

def int_label_features(train_frame, test_frame):
    fit_matrix = pd.concat([train_frame, test_frame]).as_matrix()
    categorial_encoder = MatrixLabelEncoder().fit(fit_matrix)
    
    train_categorial = categorial_encoder.transform(train_frame.as_matrix())
    test_categorial = categorial_encoder.transform(test_frame.as_matrix())
    
    return (train_categorial, test_categorial, categorial_encoder)

def frame_to_matrix_labeled(
    train_frame,
    test_frame,
    train_labels,
    test_labels,
    numeric_features,
    categorial_features):
    """ Функция преобразует фрейм к sparse матрице.
        Масштабирует вещественные признаки и кодирует категориальные целыми числами. """
    
    # Масштабируем вещественные признаки
    train_numeric, test_numeric, scaler = scale_features(
        train_frame[numeric_features],
        test_frame[numeric_features])
    
    # Закодируем категориальные признаки значениями от 0 до n с помощью MatrixLabelEncoder
    train_categorial, test_categorial, categorial_encoder = int_label_features(
        train_frame[categorial_features],
        test_frame[categorial_features])
    
    y_train = train_labels.as_matrix().flatten()
    y_test = test_labels.as_matrix().flatten()
    
    return (hstack([train_numeric, train_categorial]),
            hstack([test_numeric, test_categorial]),
            y_train,
            y_test,
            scaler,
            categorial_encoder)

def model_builder(frame, labels, numeric_features, categorial_features):
    return stratifiedKFold_fscore(
        frame,
        labels,
        GradientBoostingClassifier,
        process_frame_base,
        frame_to_matrix_labeled,
        numeric_features,
        categorial_features,
        predict_model_proba,
        seed)

Загрузим train и test dataset-ы.

In [7]:
churn_data_frame = pd.read_csv("..\..\Data\churn_data_train.csv", ",", dtype= { "Var73": np.float64 })
churn_labels_frame = pd.read_csv("..\..\Data\churn_labels_train.csv", dtype= { "labels": np.int64 })
test_data_frame = pd.read_csv("..\..\Data\churn_data_holdout.csv", ",", dtype= { "Var73": np.float64 })
test_labels_frame = pd.read_csv("..\..\Data\churn_labels_holdout.csv", dtype= { "labels": np.int64 })
print(churn_data_frame.shape, test_data_frame.shape)
print(churn_labels_frame.shape, test_labels_frame.shape)

(27999, 230) (12001, 230)
(27999, 1) (12001, 1)


Выделим числовые и категориальные признаки.

In [8]:
numeric_columns = churn_data_frame.columns[:first_categorial_index]
categorial_columns = churn_data_frame.columns[first_categorial_index:]

Предобработка фрейма

In [9]:
preprocessed_train, train_labels, preprocessed_test, test_labels, dropped_num_cols, dropped_cat_cols = process_frame_base(
    churn_data_frame,
    churn_labels_frame,
    test_data_frame,
    test_labels_frame,
    numeric_columns,
    categorial_columns)

clear_num_columns = numeric_columns.drop(dropped_num_cols)
clear_cat_columns = categorial_columns.drop(dropped_cat_cols)

preprocessed_train.shape, train_labels.shape

((27999, 211), (27999, 1))

Выделим 21 лучший признак

In [10]:
labeled_x, px, l_y, py, ne, ce = frame_to_matrix_labeled(
    preprocessed_train,
    preprocessed_test,
    train_labels,
    test_labels,
    clear_num_columns,
    clear_cat_columns)
preprocessed_y = train_labels.as_matrix().flatten()
kbest_mdl = SelectKBest(k=labeled_x.shape[1]).fit(labeled_x, preprocessed_y)
k_best_21_features = [f[0] for f in sorted(zip(preprocessed_train.columns, kbest_mdl.scores_), key=lambda f: -f[1])][:21]
k_best_21_num = pd.Index(list(set(numeric_columns) & set(k_best_21_features)))
k_best_21_cat = pd.Index(list(set(categorial_columns) & set(k_best_21_features)))
print("Best numeric:", ", ".join(k_best_21_num))
print("Best categorial:", ", ".join(k_best_21_cat))

Best numeric: Var72, Var144, Var74, Var126, Var189, Var65, Var13, Var7, Var73, Var113, Var81
Best categorial: Var227, Var193, Var205, Var229, Var210, Var218, Var228, Var216, Var211, Var207


Преобразуем фреймы к матрицам 

In [11]:
preprocessed_x_train, preprocessed_x_test, preprocessed_y_train, preprocessed_y_test, ne, ce = frame_to_matrix_labeled(
    preprocessed_train[k_best_21_features],
    preprocessed_test[k_best_21_features],
    train_labels,
    test_labels,
    k_best_21_num,
    k_best_21_cat)

preprocessed_x_test.shape, preprocessed_y_test.shape

((12001, 21), (12001,))

Построим модель

In [12]:
model = GradientBoostingClassifier().fit(preprocessed_x_train, preprocessed_y_train)

Посчитаем вероятности принадлежности тестовых данных к классу

In [13]:
probabilities = predict_model_proba(preprocessed_x_test, model)

In [15]:
def calculate_revenue(probas, threshold, recall, churn_size):
    clients_count = len(probas)
    churn_count = np.round(clients_count * churn_size, 0) # Количество пользователей, которые собриаются уйти
    churn_found = np.round(churn_count*recall, 0) # Количество пользователей, собирающихся уйти, которых выявила модель
    participants_count = len([p for p in probas if p > threshold]) # Количество пользователей в кампании
    not_participants_cnt = clients_count-participants_count # Количество пользователей не участвующих в кампании
    others_count = participants_count - churn_found # Количество пользователей не собирающихся уходить
    
    retention_size = np.round(churn_found * retention_probability, 0) # Количество пользователей, которых удалось сохранить
    churned = churn_count-retention_size # Количество пользователей, которые ушли
    saved_count = clients_count - churned # Количество пользователей после проведения кампании
    
    month_revenue = saved_count*month_revenue_per_user # Сколько денег получили в месяц
    money_spent = participants_count * price_per_user_churn # Сколько денег потратили на удержание в месяц
    
    month_revenue_default = (clients_count - churn_count) * month_revenue_per_user # Сколько денег получили бы если не проводили
                                                                                   # кампанию.
    campaign_revenue = month_revenue - money_spent
    return threshold, np.round(participants_count/clients_count*100, 2), campaign_revenue-month_revenue_default, retention_size

def build_revenue_table(shift = 0):
    data = []
    for threshold in np.arange(0.0, 1.0, 0.01):
        predictions = [-1 if probability < threshold else 1 for probability in probabilities]
        recall = recall_score(list(preprocessed_y_test), predictions) + shift
        revenue_info = calculate_revenue(probabilities, threshold, recall, churn_size)
        data.append(revenue_info)
    return data

In [16]:
revenue_table = build_revenue_table()
print(pd.DataFrame(
    sorted([rd for rd in revenue_table if rd[2] > 0], key=lambda r: -r[3]),
    columns=["Threshold", "Clients %", "Effect", "Saved clients count"]).head(5))
print()
print(pd.DataFrame(
    sorted(revenue_table, key=lambda r: -r[2]),
    columns=["Threshold", "Clients %", "Effect", "Saved clients count"]).head(5))

   Threshold  Clients %  Effect  Saved clients count
0       0.12      14.78   490.0                312.0
1       0.13      12.80   900.0                286.0
2       0.14      11.35  1050.0                262.0
3       0.15       9.87  1215.0                238.0
4       0.16       8.61  1105.0                209.0

   Threshold  Clients %  Effect  Saved clients count
0       0.22       4.03  1510.0                131.0
1       0.20       5.24  1445.0                153.0
2       0.21       4.64  1445.0                141.0
3       0.23       3.61  1435.0                120.0
4       0.24       3.21  1315.0                108.0


Из результатов расчета видно, что можно применить две стратегии: сохранение максимального количества клиентов и получение максимальной прибыли.
Исходя из этого можно выбрать оптимальное количество участников кампании.
Для моей модели оптимальным оказалось взять 4.05% клиентов склонных к оттоку (по мнению модели) для получения максимальной прибыли и 14.78% клиентов склонных к оттоку для сохранения максимального количества клиентов.
Будем смотреть оптимальный размер топа для получения максимальной прибыли

В своей весьма простой экономической модели я ввел следующие параметры
- сколько денег в среднем приносит один пользователь в месяц;
- сколько денег в среднем вы будете вкладывать в удержание одного пользователя;
- с какой вероятностью пользователь примет ваше предложение;
- порог вероятности принадлежности классу "отток" по достижении которого пользователь участвует в кампании
- предположительный процент пользователей, собирающихся уйти

### Варьируем стоимость удержания

In [18]:
data = []
for churn_price in range(5, 15):
    price_per_user_churn = churn_price # Сколько денег будем вкладывать в удержание пользователя
    revenue_table = build_revenue_table()
    data.append(sorted(revenue_table, key=lambda r: -r[2])[0])
price_per_user_churn = 5 # Сколько денег будем вкладывать в удержание пользователя (5 тугриков)
pd.DataFrame(
    data,
    columns=["Threshold", "Clients %", "Effect", "Saved clients count"],
    index=range(5, 15))

,Threshold,Clients %,Effect,Saved clients count
5,0.22,4.03,1510.0,131.0
6,0.22,4.03,1026.0,131.0
7,0.25,2.77,616.0,98.0
8,0.37,0.70,288.0,32.0
9,0.37,0.70,204.0,32.0
10,0.37,0.70,120.0,32.0
11,0.47,0.17,50.0,9.0
12,0.47,0.17,30.0,9.0
13,0.47,0.17,10.0,9.0
14,0.87,0.00,0.0,0.0


Получили в общем-то очевидный результат. Чем дороже обходится удержание клиента, тем менее выгодно проводить кампанию. Здесь я думаю играет роль не абсолютное значение стоимости удержания, а отношение стоимости удержания к прибыли, которую приносит клиент. Также заметно, что с увеличением стоимости удержания клиента выгоднее уменьшать количество участников кампании.

### Варьируем вероятность принятия предложения

In [19]:
data = []
for ret_proba in np.arange(0.24, 0.5, 0.01):
    retention_probability = ret_proba # Вероятность, с которой пользователь, собирающийся уйти, примет предложение
    revenue_table = build_revenue_table()
    data.append(sorted(revenue_table, key=lambda r: -r[2])[0])
retention_probability = 0.8 # Вероятность, с которой пользователь, собирающийся уйти, примет предложение
pd.DataFrame(
    data,
    columns=["Threshold", "Clients %", "Effect", "Saved clients count"],
    index=np.arange(0.24, 0.5, 0.01)).head(10)

,Threshold,Clients %,Effect,Saved clients count
0.24,0.87,0.00,0.0,0.0
0.25,0.52,0.10,0.0,2.0
0.26,0.61,0.04,5.0,1.0
0.27,0.61,0.04,5.0,1.0
0.28,0.61,0.04,5.0,1.0
0.29,0.61,0.04,5.0,1.0
0.30,0.57,0.07,15.0,2.0
0.31,0.57,0.07,15.0,2.0
0.32,0.47,0.17,20.0,4.0
0.33,0.47,0.17,20.0,4.0


С одной стороны мы опять получили ожидаемый результат - чем меньше вероятность принятия предложения, тем меньше эффект кампании. С другой стороны мы видим, что даже при весьма низкой вероятности принятия предложения кампания остаётся выгодной (Хотя эффект такой кампании конечно минимален).

## 4.
Всегда ли применение модели экономически оправданно? Приведите пример набора значений параметров, при которых применение модели перестает быть оправданным.

Применение модели экономически оправдано далеко не всегда. Например при стоимости удержания клиента 46% от прибыли, которую он приносит, и выше выгоднее не проводить никакой кампании.
При значениях вероятности принятия клиентом предложения ниже 25% проведение кампании также становится невыгодным.

## 5.
Оцените изменение экономического эффекта от проведения кампании по удержанию при увеличении качества модели на 1%? На 3%? При ответе на вопрос укажите, по какой метрике вы оцениваете качество.

Я измеряю качество по метрике recall. Она характеризует на какой доле клиентов класса "отток" алгоритм срабатывает. Это значит, что чем выше recall, тем больше клиентов класса "отток" поучаствует в кампании.

In [20]:
revenue_effects = sorted([r[2] for r in build_revenue_table()], key=lambda e: -e)
revenue_effects_1 = sorted([r[2] for r in build_revenue_table(0.01)], key=lambda e: -e) # recall на 1% выше
revenue_effects_3 = sorted([r[2] for r in build_revenue_table(0.03)], key=lambda e: -e) # recall на 3% выше

In [21]:
pd.DataFrame(
    list(zip(revenue_effects, revenue_effects_1, revenue_effects_3)),
    columns=["Model Recall", "1% better", "3% better"]).head(1)

,Model Recall,1% better,3% better
0,1510.0,1720.0,2170.0


In [22]:
percents_improvement_1 = revenue_effects_1[0]*100/revenue_effects[0] - 100
percents_improvement_3 = revenue_effects_3[0]*100/revenue_effects[0] - 100
percents_improvement_1, percents_improvement_3

(13.907284768211923, 43.708609271523187)

Улучшение полноты модели на 1% добавляет к экономическому эффекту 14%. Улучшение полноты модели на 3% добавляет к экономическому эффекту 44% (На другой машине цифры могут немного отличаться)

## 6.
Как вы думаете, является ли экономически оправданным вложение средств в улучшение качества модели? На сколько нужно улучшить модель, чтобы это качественно сказалось на экономическом эффекте от удержания?

На мой взгляд, на данном уровне модели, вложение средств в улучшение качества модели является экономически выгодным.